In [97]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import os
import requests
import time
import pandas as pd
import re
from selenium.webdriver.chrome.options import Options

In [35]:
driver.switch_to.window(driver.window_handles[0])

In [57]:
#creating input dictionary
data = {'input_ids':[]}

In [98]:
class link:
    def driver_start(url):
        options = Options()
        options.add_argument("--headless")
        driver = webdriver.Chrome(options = options)
        driver.get(url)
        return driver
        
    def extract_year_links():
        year_links = []
        driver = driver_start('https://indiankanoon.org/browse/chennai/')
        table = driver.find_element(By.TAG_NAME, 'table')
        rows = table.find_elements(By.TAG_NAME, 'tr')
        for row in rows:
            tds = row.find_elements(By.TAG_NAME, 'td')
            for td in tds:
                a_tag = td.find_element(By.TAG_NAME, 'a')
                link = a_tag.get_attribute('href')
                year_links.append(link)
        driver.quit()
        return year_links

    year_links = extract_year_links()
    
    def extract_page_links(year_links):
        page_links = []
        for i in year_links:
            driver = driver_start(i)
            row = driver.find_element(By.TAG_NAME, 'tr')
            td = row.find_element(By.TAG_NAME, 'td')
            a_tag = td.find_element(By.TAG_NAME, 'a')
            page_links.append(a_tag.get_attribute('href'))
        driver.quit()
        return page_links

    page_links = extract_page_links(year_links)


            
link_obj = link
links = link_obj.page_links
print(links)
print(len(links))

TimeoutException: Message: timeout: Timed out receiving message from renderer: 300.000
  (Session info: chrome=128.0.6613.119)
Stacktrace:
	GetHandleVerifier [0x00007FF6E36BB5D2+29090]
	(No symbol) [0x00007FF6E362E689]
	(No symbol) [0x00007FF6E34EB1CA]
	(No symbol) [0x00007FF6E34D3F0D]
	(No symbol) [0x00007FF6E34D3C10]
	(No symbol) [0x00007FF6E34D1AD3]
	(No symbol) [0x00007FF6E34D231F]
	(No symbol) [0x00007FF6E34E0C1E]
	(No symbol) [0x00007FF6E34F95CD]
	(No symbol) [0x00007FF6E34FF16A]
	(No symbol) [0x00007FF6E34D2A6A]
	(No symbol) [0x00007FF6E34F943F]
	(No symbol) [0x00007FF6E35864BB]
	(No symbol) [0x00007FF6E3566493]
	(No symbol) [0x00007FF6E35309B1]
	(No symbol) [0x00007FF6E3531B11]
	GetHandleVerifier [0x00007FF6E39D8C5D+3295277]
	GetHandleVerifier [0x00007FF6E3A24843+3605523]
	GetHandleVerifier [0x00007FF6E3A1A707+3564247]
	GetHandleVerifier [0x00007FF6E3776EB6+797318]
	(No symbol) [0x00007FF6E363980F]
	(No symbol) [0x00007FF6E36353F4]
	(No symbol) [0x00007FF6E3635580]
	(No symbol)

In [95]:
driver.quit()

In [45]:
def driver_start():
    #start the driver
    # webdriver - opens browser automatically and opens the website we provide
    driver = webdriver.Chrome()
    url = 'https://indiankanoon.org/doc/107707945/'
    driver.get(url)
    
#extracting p and blockquote tags
def extracting_p_and_blockquote_tags():
    all_tags = driver.find_element(By.XPATH, '//div[@class = "judgments"]' )
    tags = all_tags.find_elements(By.TAG_NAME, "*")
    req_tags = []
    for tag in tags:
        if tag.tag_name == 'p' or tag.tag_name == 'blockquote':
            req_tags.append(tag)
    return req_tags

#remove_starting_nums
def remove_starting_nums(str):
    return re.sub(r'^\d+.', '', str)

def remove_starting_nums_check(tagname, ):
    txt = ''
    for i in req_tags:
        if i.tag_name == 'blockquote':
            txt += i.text + "\n"
        elif i.tag_name == 'p':
            s = remove_starting_nums(i.text)
            txt += s + "\n"
            
#Extracting input_ids of each case
def extract_input_id(req_tags):
    req_tags = req_tags
    input_id = 'Give a judgement based on the following.\nCase Facts and Issues:\n'
    titles = ["Fact", "Issue"]
    for tag in req_tags:
        if tag.get_attribute('title') in titles:
            if tag.tag_name == 'blockquote':
                input_id += tag.text + "\n"
            elif tag.tag_name == 'p':
                s = remove_starting_nums(tag.text)
                input_id += s + "\n"
                input_id += tag.text + "\n"
    return input_id

#Extracting input_ids of each case
def extract_label(req_tags):
    req_tags = req_tags
    label = 'Judgement:\n'
    titles = ["Conclusion"]
    for tag in req_tags:
        if tag.get_attribute('title') in titles:
            if tag.tag_name == 'blockquote':
                label += tag.text + "\n"
            elif tag.tag_name == 'p':
                #s = remove_starting_nums(tag.text)
                #label += s + "\n"
                label += tag.text + "\n"
    return label

def apply_chat_format(input_id, label):
    chat_template_applied_input_id= f"<|system|>\nYou are a helpful Indian law assistant.<|end|>\n<|user|>\n{input_id}<|end|>\n<|assistant|>\n{label}<|end|>"
    return chat_template_applied_input_id

def adding_input_id_to_dictionary(chat_template_applied_input_id):
    data['input_ids'].append(chat_template_applied_input_id)
    return data
    
def main():
    driver_start()
    req_tags = extracting_p_and_blockquote_tags()
    input_id = extract_input_id(req_tags)
    label = extract_label(req_tags)
    chat_template_applied_input_id = apply_chat_format(input_id, label)
    data = adding_input_id_to_dictionary(chat_template_applied_input_id)
    print(input_id, "\n")
    print("---------------------------------------------------------------------------------")
    print("\n", label)

In [ ]:
driver.quit()

In [35]:
data = adding_input_id_to_dictionary(chat_template_applied_input_id)

In [36]:
print(data)

{'input_ids': ['<|system|>\nYou are a helpful Indian law assistant.<|end|>\n<|user|>\nGive a judgement based on the following.\nCase Facts and Issues:\nW.A.Nos.849 to 854 of 2010 & 2066 of 2010 \x16 Whether the Appellants, who were part of Special Task Force and were given "one stage accelerated promotion" by executive orders, can claim consequential seniority in the accelerated promotional post over their senior lower category; whether the learned single Judge was right in holding that G.O.Ms.1396 Home (Pol.1A) Department dated 03.10.2007 sets right the ambiguity regarding seniority position in accordance with General Rule 35 of Tamil Nadu State and Subordinate Service Rules and whether the Order of the learned single Judge warrants any interference, are the points falling for consideration in these intra-Court appeals. Validity of G.O.Ms.1396 Home Department dated 03.10.2007 is the core issue.\nW.A.Nos.849 to 854 of 2010 & 2066 of 2010 \x16 Whether the Appellants, who were part of Sp

In [146]:
with open('scrape_dataset/label.txt', 'w', encoding = 'utf8') as f:
    f.write(label)
f.close()

In [134]:
with open('scrape_dataset/input_id.txt', 'w', encoding = 'utf8') as f:
    f.write(input_id)
f.close()

In [118]:
inputs_df['input_ids'] = input_ids

In [120]:
inputs_df.to_csv('scrape_dataset/input_ids.txt', encoding = 'utf8', sep = '\n', index = False)

In [13]:
driver.quit()